In [ ]:
import org.apache.spark._
import org.apache.spark.streaming._
import org.apache.spark.streaming.StreamingContext._ 
import org.apache.spark.sql.SparkSession

// Creamos la configuracion
val conf = new SparkConf().setMaster("local[4]").setAppName("CuentaPalabras").set("spark.driver.allowMultipleContexts","true")
// Creamos el Streaming Context
val ssc = new StreamingContext(conf, Seconds(10))

// Creamos un socket
val lines = ssc.socketTextStream("localhost", 9999)

val words = lines.flatMap(_.split(" "))


words.foreachRDD { rdd =>

  // Get the singleton instance of SparkSession
  val spark = SparkSession.builder.config(rdd.sparkContext.getConf).getOrCreate()
  import spark.implicits._

  // Convert RDD[String] to DataFrame
  val wordsDataFrame = rdd.toDF("word")

  // Create a temporary view
  wordsDataFrame.createOrReplaceTempView("words")

  // Do word count on DataFrame using SQL and print it
  val wordCountsDataFrame = 
    spark.sql("select word, count(*) as total from words group by word")
  wordCountsDataFrame.show()
}

ssc.start()             
ssc.awaitTerminationOrTimeout(60000)